In [14]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

# Homework 4

(In the following set $\alpha = 0.05$, $M=500$)

For the first exercise you probably need to watch chapter 3 of the [Statistical Thinking on Python 2 course](https://campus.datacamp.com/courses/statistical-thinking-in-python-part-2/introduction-to-hypothesis-testing?ex=10).
And look up the help file for the numpy `rng.normal()` function.

1. Comparing Bootstrap and Permutation Testing
    * Generate two arrays of 50 (each) random samples from a normal distribution, each with mean $\mu_1=\mu_2=50$, but with different stdevs: $\sigma_1 = 1, \sigma_2 = 4$.
    * Based on those two samples, test the Null Hypothesis $H_0: \mu_1=\mu_2$, $H_A: \mu_1 \neq \mu_2$ in two ways: (i) Bootstrap and (ii) Permutation.
    * Repeat the previous two steps (in a loop) $500$ times and save the test decisions in an array.
    * Compute the type-I error rates for the two test procedures. Discuss your findings.

2. Early Stopping
    * Simulate a **biased** coin that imitates Web clicks: With probability $p=0.1$ you generate $x=1$ (yes click), and with $p=0.9$ we have $x=0$ (no click). Flip the coin $400$ times and repeat the entire sequence. (Prepare to repeat this process $M$ times).
    * Write a function that tests the Null Hypothesis $H_0: p \leq 0.05$, $H_A: p > 0.05$ by applying this rule <br><br>
    $t = \frac{\sqrt{n} \cdot (\hat{p}- 0.05)}{\sqrt{0.05 \cdot 0.95}} > z_{1- \alpha}$ <br> where $\hat{p} = \sum x/n$ is the observed proportion of "ones" up to time step $n$.
    * Compute the type-II error rates for time steps $50$, $100$ and $400$.
    * Wouldn't it be nice to deploy an "early stopping" rule ? Try out the following strategy: stop the test **at any time** if your test statistic exceeds the threshold $z_{1- \alpha}$!
        * Compute your type-I error at the end of the $400$ time steps and compare to the one you set ($\alpha$).




# Bootstrap (I)

In [48]:
rng = np.random.default_rng()
bootstrap_decisions = np.empty(500, dtype=bool)

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""
    bs_replicates = np.empty(size)
    for i in range(size):
        # Generate a bootstrap sample
        bs_sample = np.random.choice(data, size=len(data))
        # Compute the statistic and store it
        bs_replicates[i] = func(bs_sample)
    return bs_replicates

# test for H0
loop_for = 500# Number of times to repeat the test
alpha = 0.05  


for i in range(loop_for):
    # generating bootstrap replicates
    dist1 = rng.normal(50, 1, size=50)
    dist2 = rng.normal(50, 4, size=50)

    # Calculate the observed difference for this experiment
    observed_diff = np.mean(dist1) - np.mean(dist2)
    
    bs_reps1 = draw_bs_reps(dist1, np.mean, size=500)
    bs_reps2 = draw_bs_reps(dist2, np.mean, size=500)

    # taking the diffrence between means of bootstrap replicates
    bs_reps_diff = bs_reps1 - bs_reps2

    # determine and set confidence intervals 
    conf_int = np.percentile(bs_reps_diff, [(alpha/2)*100, 100-(alpha/2)*100])
    lower_ci = conf_int[0]
    upper_ci = conf_int[1]
    
    # If 0 is outside the interval, we reject H0.
    if 0 < lower_ci or 0 > upper_ci:
        bootstrap_decisions[i] = True  # Reject H0
    else:
        bootstrap_decisions[i] = False # Fail to reject H0

type_I_error_bootstrap = np.mean(bootstrap_decisions)
print("Bootstrap Type I Error Rate:",type_I_error_bootstrap, "%. Target was:", alpha)
        
'''
num_bins = int(np.sqrt(len(bs_reps_diff)))
sns.set()
plt.hist(bs_reps_diff, bins=num_bins, edgecolor='black');
plt.xlabel('Difference of means')
plt.ylabel('Frequency (count)')
plt.show()
'''

Bootstrap Type I Error Rate: 0.062 %. Target was: 0.05


"\nnum_bins = int(np.sqrt(len(bs_reps_diff)))\nsns.set()\nplt.hist(bs_reps_diff, bins=num_bins, edgecolor='black');\nplt.xlabel('Difference of means')\nplt.ylabel('Frequency (count)')\nplt.show()\n"

In [37]:
# (II) Permutation

dist_combined = np.concatenate((dist1, dist2))
scrambled_data = np.random.permutation(dist_combined)

perm_samples = np.split(scrambled_data, 2)

perm_sample_1 = perm_samples[0]
perm_sample_2 = perm_samples[1]